In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm


import pandas as pd
import numpy as np
import requests
import time
import re

In [11]:
df = pd.read_csv('kakao_영등포구맛집 (1).csv')

In [12]:
df.drop(['Unnamed: 0', 'Unnamed: 0.1'], inplace=True, axis=1)
df.head()

,ID,stores,X,Y,road_address,place_url
0,20908598,공차 영등포구청점,126.897061,37.526699,서울 영등포구 당산로 136,http://place.map.kakao.com/20908598
1,1714644708,이도숙성회,126.896237,37.526695,서울 영등포구 당산로31길 4,http://place.map.kakao.com/1714644708
2,27527233,까치네분식,126.896689,37.527085,서울 영등포구 당산로 139,http://place.map.kakao.com/27527233
3,22246273,푸른농장 해바라기생고기정육식당 영등포점,126.896155,37.527045,서울 영등포구 당산로33길 5,http://place.map.kakao.com/22246273
4,19609040,사조참치 영등포구청점,126.896554,37.526706,서울 영등포구 당산로 133,http://place.map.kakao.com/19609040


In [14]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

/var/folders/6b/s1063_qj51s2xh9qm5l7ntq40000gn/T/ipykernel_3692/2928635203.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [16]:
# 수집할 정보

menu_price = []
category = []
business_hours = [] 
rating = []
review_count = []
phone_number = [] 
last_update = []
delay = 0.1

In [18]:
for x in tqdm(df['place_url']):
    driver.get(x)
    time.sleep(1)
    try:
        business_hours.append(driver.find_element(By.CLASS_NAME, 'txt_operation').text) # 영업시간
        time.sleep(delay)
    except NoSuchElementException:
        business_hours.append('-')
        time.sleep(delay)
    
    try:
        # 같은 class를 공유해 작성되는 다른 요소가 있으나 제일 첫번째 요소에 해당되어 별도 처리는 하지 않았음
        rating.append(driver.find_elements(By.CLASS_NAME, 'color_b'))
        time.sleep(delay)
    except NoSuchElementException:
        rating.append('-')
        time.sleep(delay)
    
    try:
        category.append(driver.find_element(By.CLASS_NAME, 'txt_location').text) # 카테고리
        time.sleep(delay)
    except NoSuchElementException:
        category.append('-')
        time.sleep(delay)
    
    try:
        # 같은 class를 공유해 생성되는 음식점이 있어 if를 통해 요소가 3개 이상 수집된 경우 처리
        p = re.compile('[a-zA-Z]+')
        review_temp = driver.find_elements(By.CLASS_NAME, 'color_g')
        # review_temp[0]이 영문으로 되어있으면 [1]번을 정제해서 추가하고 반복문 탈출 아니면 [0]번을 추가
        for x in review_temp:
            if p.match(x.text) == None :
                review_count.append(re.sub('[(0-9)]', '', x.text))
                time.sleep(delay)
                break
            else:
                review_count.append(re.sub('[(0-9)]', '', x.text))
                time.sleep(delay)
            
    except NoSuchElementException:
        review_count.append('-')
        time.sleep(delay)
    
    try:
        phone_number.append(driver.find_element(By.CLASS_NAME, 'txt_contact').text) # 대표번호
        time.sleep(delay)
    except NoSuchElementException:
        phone_number.append('-')
        time.sleep(delay)
    
    try:
        last_update.append(driver.find_element(By.CLASS_NAME, 'date_revise').text) # 최종 업데이트 날짜
        time.sleep(delay)
    except NoSuchElementException:
        last_update.append('-')
        time.sleep(delay)
    
    # 메뉴 펼치기
    try:
        driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[3]/a/span[1]').click()
        time.sleep(delay)
    except NoSuchElementException:
        pass
    
    menu_temp = []
    price_temp = []
    # photo_menu 확인
    photo_menues = driver.find_elements(By.CLASS_NAME, 'photo_type')
    if len(photo_menues) == 0:
        pass
    else :
        for y in photo_menues:
            y = y.text.split('\n')
            try:
                menu_temp.append(y[0])
                price_temp.append(y[1])
            except:
                if y[0] != None: # 메뉴는 존재하지만 가격이 없는 경우 공란으로 처리
                    menu_temp.append(y[0])
                    price_temp.append('-')
                else:
                    pass
                
    time.sleep(delay)
            
    normal_menues = driver.find_elements(By.CLASS_NAME, 'nophoto_type')
    if len(normal_menues) == 0:
        pass
    else :
        for z in normal_menues:
            z = z.text.split('\n')
            try:
                menu_temp.append(y[0])
                price_temp.append(y[1])
            except:
                if y[0] != None: # 메뉴는 존재하지만 가격이 없는 경우 공란으로 처리
                    menu_temp.append(y[0])
                    price_temp.append('-')
                else:
                    pass
    # key가 존재하면 생성하고 아니면 공란으로 대체
    menu_price.append({ '' if key == None else key : value for key, value in zip(menu_temp, price_temp)}) 
    time.sleep(1)

  0%|          | 0/1412 [00:01<?, ?it/s]


AttributeError: 'list' object has no attribute 'text'

In [ ]:
driver.close()

In [ ]:
df['menu_price'] = menu_price
df['category'] = category
df['business_hours'] = business_hours 
df['rating'] = rating
df['review_count'] = review_count
df['phone_number'] = phone_number
df['last_update'] = last_update
df.to_csv('카카오API_크롤링(selenium).csv', encoding='utf-8-sig', index=False)

ValueError: Length of values (12) does not match length of index (1412)

In [21]:
df.tail(20)

,ID,stores,X,Y,road_address,place_url
1392,77876642,잠수교집 여의도점,126.931048,37.526225,서울 영등포구 여의나루로 117,http://place.map.kakao.com/77876642
1393,12806277,호우양꼬치 여의나루점,126.931055,37.526230,서울 영등포구 여의나루로 117,http://place.map.kakao.com/12806277
1394,21332784,농부와닭동네,126.931086,37.526241,서울 영등포구 여의나루로 117,http://place.map.kakao.com/21332784
1395,61565227,잉쓰,126.931055,37.526205,서울 영등포구 여의나루로 117,http://place.map.kakao.com/61565227
1396,1896148689,순수치킨 아라호점,126.935080,37.528460,서울 영등포구 여의동로 338,http://place.map.kakao.com/1896148689
1397,10761242,다래횟집,126.931091,37.526241,서울 영등포구 여의나루로 117,http://place.map.kakao.com/10761242
1398,789388227,미국가주우육면대왕,126.931053,37.526230,서울 영등포구 여의나루로 117,http://place.map.kakao.com/789388227
1399,117377026,통영굴찜,126.931053,37.526230,서울 영등포구 여의나루로 117,http://place.map.kakao.com/117377026
1400,142013749,코브라독스 푸드트럭,126.933595,37.526348,서울 영등포구 여의동로 330,http://place.map.kakao.com/142013749
1401,36727652,저팔계마라탕,126.931055,37.526230,서울 영등포구 여의나루로 117,http://place.map.kakao.com/36727652
